In [260]:
from typing import (
    Optional,
    Tuple,
    Union,
)

import ipytest
import numpy as np
import pytest
import torch
from torch import nn
from torch.nn.common_types import Tensor

In [263]:
ipytest.autoconfig()

In [314]:
def convolution2D(
    input: Tensor,
    weights: Tensor,
    bias: Optional[Tensor] = None,
    stride: Optional[Union[int, Tuple]] = 1,
    padding: Optional[Union[int, Union[Tuple, str]]] = 0,
    dilation: Optional[Union[int, Tuple]] = 1,
    groups: Optional[int] = 1,
) -> Tensor:
    batch_size, in_channels, input_height, input_width = input.shape
    out_channels, in_channels_groups, weights_height, weights_width = weights.shape

    input = nn.functional.pad(input, (padding, padding, padding, padding))

    result_height = (
        input_height + 2 * padding - dilation * (weights_height - 1) - 1
    ) // stride + 1
    result_width = (
        input_width + 2 * padding - dilation * (weights_width - 1) - 1
    ) // stride + 1

    grouped_channels = out_channels // groups if groups else out_channels

    result = torch.zeros((batch_size, grouped_channels, result_height, result_width))

    for batch in range(batch_size):
        for channel in range(out_channels):
            for i in range(0, input.shape[2] - dilation*(weights_height - 1) + 1, stride):
                for j in range(
                    0, input.shape[3] - dilation*(weights_width - 1) + 1, stride
                ):
                    for group in range(grouped_channels):
                        d = input[
                            batch, :, i : i + weights_height, j : j + weights_width
                        ]
                        result[batch, group, i // stride, j // stride] = (
                            d * weights[channel]
                        ).sum()
            result[batch] += bias[channel] if bias else 0

    return result

In [315]:
@pytest.fixture(scope='class')
def inputs():
    return torch.randn(1, 2, 4, 4)

@pytest.fixture(scope='class')
def weights():
    return torch.randn(1, 2, 3, 3)

@pytest.fixture(scope='class')
def bias():
    return torch.randn(1)

In [316]:
%%ipytest

@pytest.mark.usefixtures('inputs')
@pytest.mark.usefixtures('weights')
@pytest.mark.usefixtures('bias')
class TestConv2D:
    def test_conv2d_success(self, inputs, weights):
        result = convolution2D(inputs, weights)
        expected_result = nn.functional.conv2d(inputs, weights)
        assert torch.allclose(expected_result, result)
        
    def test_conv2d_bias_success(self, inputs, weights, bias):
        result = convolution2D(inputs, weights, bias)
        expected_result = nn.functional.conv2d(inputs, weights, bias)
        assert torch.allclose(expected_result, result)
        
    def test_conv2d_bias_padding_success(self, inputs, weights, bias):
        result = convolution2D(inputs, weights, bias, padding=5)
        expected_result = nn.functional.conv2d(inputs, weights, bias, padding=5)
        assert torch.allclose(expected_result, result)
        
    def test_conv2d_bias_padding_stride_success(self, inputs, weights, bias):
        result = convolution2D(inputs, weights, bias, padding=5, stride=2)
        expected_result = nn.functional.conv2d(inputs, weights, bias, padding=5, stride=2)
        assert torch.allclose(expected_result, result)
        
    def test_conv2d_bias_padding_stride_dilation_success(self, inputs, weights, bias):
        result = convolution2D(inputs, weights, bias, padding=5, stride=2, dilation=2)
        expected_result = nn.functional.conv2d(inputs, weights, bias, padding=5, stride=2, dilation=2)
        assert torch.allclose(expected_result, result)

FFFFF                                                                                        [100%]
============================================ FAILURES =============================================
_________________________________ TestConv2D.test_conv2d_success __________________________________

self = <__main__.TestConv2D object at 0x000002430FDFEED0>
inputs = tensor([[[[-0.7242,  0.5520,  1.7713,  0.9649],
          [-0.4341, -0.8966,  0.1245,  0.0645],
          [ 0.0583, -0...,  1.3643,  0.4532],
          [-1.3859, -0.4927,  0.9672, -0.6518],
          [-2.3218, -1.5389, -1.1958,  0.3439]]]])
weights = tensor([[[[ 0.0609, -0.2714, -0.6916],
          [ 0.3006,  0.7230, -0.6429],
          [-0.3125, -0.4023, -1.2953]],

         [[ 0.5244, -0.8564, -0.3703],
          [-2.5459, -1.3926, -0.4236],
          [-1.9287, -0.8121, -0.9855]]]])

    def test_conv2d_success(self, inputs, weights):
>       result = convolution2D(inputs, weights)

C:\Users\user\AppData\Local\Temp\ipykern